# Coronavirus Reverse Engineering

## Protein Recognition and Classification

In a previous [notebook](./coronaversing.ipynb) we did an exploratory analysis of the data we collected from online sources and created a database of regions for each genome. The portions of the genome for each sample is classified into 3 types: ORF (a potential coding frame), UTR (untranslated regions at the 5' and 3' ends of the genome) and ING (intergenic region, located between 2 ORFs). Each portion has a label assigned, for example an UTR region could be "3utr" or "5utr" and ORFs could be "S", "N" or "ORF1ab", and so on. The idea is to have a standardized mechanism to access regions of the genome common to all coronavirus samples to perform large scale analysis efficiently.

At this point, the labels for each region are left "undefined", since we need to execute a detailed analysis of each frame to recognize what is coding for. For example, given a coronavirus genome and the list of ORFs, how do we know which one is the ORF1a and how to join it with ORF1b to get ORF1ab? These are (usually) the first two ORFs that combined make up two thirds of the genome. Joining them it's tricker, since we have to locate the slippery sequence and perform a -1 frame shift. Sometimes there is an ORF in the 5'UTR region and the "untranslated region" now becomes potentially translatable. 

How do we know where the structural proteins coding segment are? Coronaviruses encode three conserved membrane-associated proteins that are incorporated in virions: spike (S), envelope (E), membrane (M) and a nucleoprotein (N). These four proteins seems to occur in the order S–E–M–N but between those genes, there are species-specific accessory proteins. How do we recognize these and find common sequences to classify them within a specie?

Some of the details that we have to figure out.

**TABLE OF CONTENTS**

* [ORF Tagging Algorithm](#section1)
   * [ORF1a/ORF1ab](#section2)

In [6]:
from matplotlib import pyplot as plt

from IPython.display import display, HTML
from IPython.display import Markdown as md
from IPython.core.display import Image
import time
from sklearn.linear_model import LinearRegression

from reportlab.lib import colors
from reportlab.lib.units import cm

import matplotlib.pyplot as plt
import forgi.visual.mplotlib as fvm
import forgi

from Bio.Seq import Seq
from Bio import Entrez
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Alphabet import IUPAC
from Bio.SeqUtils import seq3
from Bio import Align

import bisect 
import pandas as pd
import seaborn as sb
import numpy as np
import seaborn as sns
from scipy import stats
import math
import zlib

from intervaltree import Interval, IntervalTree
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

import datetime
import os
import lzma as xz

# file periodically removed to update with the latest data
cache_vrs_file = ".cache_vrs.pkl" 
cache_cds_file = ".cache_cds.pkl" 
cache_regions_file = ".cache_regions.pkl" 

hidden_columns = ["sequence", "file_path"]
hidden_cds_columns = ["translation", "file_path", "location"]
hidden_regions_columns = ["id", "rna", "protein", "path"]

if os.path.isfile(cache_regions_file):
    corona_db = pd.read_pickle(cache_regions_file)
else:
    %run ./coronaversing.ipynb

# 1. ORF tagging <a id="section1"></a>

This is how the current DB looks like for a given genome (we'll use NC_045512.2 as a reference). 

In [7]:
ncov_regions = corona_db[corona_db["id"] == "NC_045512.2"]
display(HTML(ncov_regions.drop(hidden_regions_columns, axis=1).to_html()))

,type,label,date,collection,start,end,length,unknown
rid,,,,,,,,
NC_045512.2|UTR@00000,UTR,5utr,2020-03-13,Dec-2019,0,265,265,0.0
NC_045512.2|ORF@00265,ORF,undef,2020-03-13,Dec-2019,265,13483,13218,0.0
NC_045512.2|ING@13483,ING,undef,2020-03-13,Dec-2019,13483,13767,284,0.0
NC_045512.2|ORF@13767,ORF,undef,2020-03-13,Dec-2019,13767,21555,7788,0.0
NC_045512.2|ORF@21535,ORF,undef,2020-03-13,Dec-2019,21535,25384,3849,0.0
NC_045512.2|ORF@25331,ORF,undef,2020-03-13,Dec-2019,25331,25448,117,0.0
NC_045512.2|ORF@25392,ORF,undef,2020-03-13,Dec-2019,25392,26220,828,0.0
NC_045512.2|ORF@26182,ORF,undef,2020-03-13,Dec-2019,26182,26281,99,0.0
NC_045512.2|ORF@26244,ORF,undef,2020-03-13,Dec-2019,26244,26472,228,0.0


As you might guess, ORF@00265 and ORF@13767 are orf1a (~13.2kbp) and orf1b (~7.7kpb). Ideally, we would like to remove orf1b from our database and replace it with orf1ab, since that is consistent to what happens in reality : this gene is translated as orf1a and orf1ab, with 1ab resulting from a pseudoknot-induced -1 ribosomal frame shifting event at a slippery sequence of UUUAAAC at the orf1a/orf1b junction, which we will have to find to get the correct aminoacid sequence of the polyprotein which it's around ~21kbp.

The first open reading frame ORF@21535 downstream orf1ab encodes the spike (S) glycoprotein and the size it's around ~3.8 kbps. E, M and N are encoded by ORF@26244, ORF@26522 and ORF@28273 respectively. We know this because the genbank [entry](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2) at NCBI contains annotations with the name of each CDS. This annotations aren't always available, so we should develop a method to generically tag each coding region with the protein it codifies.

Besides the polyprotein and the structural proteins there are a few other coding segments which we are going to deal later.

## 1.1 ORF1a/ORF1ab <a id="section2"></a>
